# Tiền xử lý dữ liệu từ dữ liệu đã được làm sạch

    Import required library

In [26]:
import pandas as pd
import numpy as np
from pprint import pprint
import json
import re
import datetime

# Định dạng lại dữ liệu trong cột 

    Chuyển đổi chuỗi thời gian thành kiểu datetime

In [27]:
df = pd.read_csv('Data/clean_data.csv')

In [28]:
df['time'] = pd.to_datetime(df['time'])

    Chuyển đổi dữ liệu cột image và video thành kiểu boolean    

In [29]:
df['image'] = df['image'].notna()
df['video'] = df['video'].notna()

# Thêm các cột cần thiết

    Thêm cột post_length biểu thị độ dài của bài đăng

In [30]:
# Calculating the length of each post
df['post_length'] = df['post_text'].str.len()

    Thêm cột content_type xác định bài đăng chỉ chứa ảnh hoặc chỉ chứa video hoặc chứa cả 2 hoặc không chứa gì

In [31]:
# Define a function to categorize each post based on its content
def categorize_post(row):
    if row['image'] and row['video']:
        return 'Image and Video'
    elif row['image']:
        return 'Image Only'
    elif row['video']:
        return 'Video Only'
    else:
        return 'Neither'
# Apply the function to each row
df['content_type'] = df.apply(categorize_post, axis=1)

    Thêm cột hour xác định bài viết được đăng vào mấy giờ 

In [32]:
df['hour'] = df['time'].dt.hour

    Thêm cột day_of_week xác định bài viết được đăng vào thứ mấy trong tuần 

In [33]:
df['day_of_week'] = df['time'].dt.day_name()

# Lưu dataframe thành file csv 

In [34]:
df.to_csv('Data/perfect_data.csv', index = False)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   post_id         90 non-null     int64         
 1   post_text       90 non-null     object        
 2   time            90 non-null     datetime64[ns]
 3   image           90 non-null     bool          
 4   video           90 non-null     bool          
 5   comments        90 non-null     int64         
 6   shares          90 non-null     int64         
 7   reactions       90 non-null     object        
 8   reaction_count  90 non-null     int64         
 9   post_length     90 non-null     int64         
 10  content_type    90 non-null     object        
 11  hour            90 non-null     int64         
 12  day_of_week     90 non-null     object        
dtypes: bool(2), datetime64[ns](1), int64(6), object(4)
memory usage: 8.0+ KB
